# Extended Tofts model

In [ ]:
import os
import numpy
from matplotlib import pyplot
import csv
import pandas

## Background
some background on the method if necessary

## Test data
Explain data that is being used
Tolerances

## Import data

## Results

In [4]:
# prepare data for bland-altman like plots

## Notes
Additional notes/remarks

### References